In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
from pathlib import Path
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import tensorflow as tf
from random import shuffle
from tqdm import tqdm_notebook
import itertools
import cv2
import shutil


from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

In [ ]:
gpus=tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)
print(tf.__version__)
policy = mixed_precision.Policy('mixed_float16')

#policy = mixed_precision.Policy('mixed_float16')

mixed_precision.set_policy(policy)

2.4.1
Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Tesla P100-PCIE-16GB, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [ ]:
dataset = 'Custom/'

In [ ]:
num_images = 10
img_size = 128

In [ ]:
def Xception2(input_shape=(None,None,None,3), lr = 1e-3):
    channel_axis = 3
    img_input = layers.Input(input_shape)
    x = layers.TimeDistributed(layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name='block1_conv1'))(img_input)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block1_conv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block1_conv1_act'))(x)
    x = layers.TimeDistributed(layers.Conv2D(64, (3, 3), use_bias=False, name='block1_conv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block1_conv2_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block1_conv2_act'))(x)

    residual = layers.TimeDistributed(layers.Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block2_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block2_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block2_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block2_pool'))(x)
    x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block3_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block3_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block3_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block3_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block3_pool'))(x)
    x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(728, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block4_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block4_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block4_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block4_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block4_pool'))(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv1_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv1'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv1_bn'))(x)
        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv2_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv2'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv2_bn'))(x)
        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv3_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv3'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv3_bn'))(x)

        x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block13_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block13_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block13_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block13_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(1024, (3, 3), padding='same', use_bias=False, name='block13_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block13_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool'))(x)
    x = layers.add([x, residual])

    x = layers.TimeDistributed(layers.SeparableConv2D(1536, (3, 3), padding='same', use_bias=False, name='block14_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block14_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block14_sepconv1_act'))(x)

    x = layers.TimeDistributed(layers.SeparableConv2D(2048, (3, 3), padding='same', use_bias=False, name='block14_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block14_sepconv2_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block14_sepconv2_act'))(x)

    x = layers.TimeDistributed(layers.GlobalAveragePooling2D(name='avg_pool'))(x)
    
    model = models.Model(img_input, x, name='xception')
    model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

def get_callbacks(modelname, es_patience, rlr_patience, verbose = 1):
    checkpointer = ModelCheckpoint(modelname+'.h5', monitor = 'val_loss', save_best_only = True, verbose = verbose, mode = 'min')
    #earlystopper = EarlyStopping(monitor = 'val_loss', patience = es_patience, verbose = verbose, mode = 'min')
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 1/np.sqrt(10), patience = rlr_patience, cooldown = 1 ,verbose = verbose, mode = 'min')
    return checkpointer,reduceLR


In [ ]:
def generator(l, batch_size):
    gen = iter(itertools.cycle(l))
    while 1:
        yield [next(gen) for _ in range(batch_size)]

def get_input(dataset,split,label, folder):
    images = np.zeros((num_images,img_size,img_size,3), dtype = np.float32)
    imagespath = [f for f in os.listdir(dataset+split+label+folder+"/") if 'jpg' in f]
    #print(dataset+label+folder)
    for i in range(num_images):
        img = cv2.resize(plt.imread(dataset+split+label+folder+"/"+imagespath[i]), (img_size, img_size), interpolation=cv2.INTER_CUBIC)/255
        images[i] = img
    return [images]

def video_data_generator(dataset, mode = 'Training/', batch_size = 8):
    
    if mode == 'Testing/':
        batch_size = batch_size//4 

    assert batch_size % 2 == 0
    
    real_folders = [f for f in os.listdir(dataset+mode+"Real/") if '.ipynb' not in f]
    fake_folders = [f for f in os.listdir(dataset+mode+"Fake/") if '.ipynb' not in f]
    
    #shuffle(real_folders)
    #shuffle(fake_folders)
    
   # real_gen = generator(real_folders, batch_size = batch_size // 1)
   # fake_gen = generator(fake_folders, batch_size = batch_size // 1)
    real_gen = generator(real_folders, batch_size = batch_size // 2)#not changed becoz testing time jhol chances
    fake_gen = generator(fake_folders, batch_size = batch_size // 2)
    
    while True:

        real_folder_batch, fake_folder_batch = next(real_gen), next(fake_gen)
        batch_input = []
        batch_output = []
        for real_folder, fake_folder in zip(real_folder_batch, fake_folder_batch):
            
            batch_input += get_input(dataset,mode, 'Real/', real_folder)
            batch_input += get_input(dataset,mode, 'Fake/', fake_folder)
            batch_output += [0., 1.]
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)

        yield (batch_x, batch_y)

In [ ]:
#confusion matrix function
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
#accuracy and losses function for plotting

def summarize_diagnostics(history,epoch):
    # plot loss
    plt.subplot(121)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='val')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # plot accuracy
    plt.subplot(122)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='val')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    plt.subplots_adjust(top=0.92, bottom=0.2, left=0.0, right=2.5, hspace=0.25,
                    wspace=0.25)
    plt.show()    


    #     pyplot.subplot(213)
    # 	pyplot.title('Classification Accuracy')
    # 	pyplot.plot(history.history['accuracy'], color='blue', label='train')
    # 	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename +'InceptionNewAdam'+str(epoch)+ '_plot.png')



In [ ]:
# Video
#from tensorflow.keras.applications import Xception
model_path = "/content/drive/My Drive/All_Models/" # Save models here
Path(model_path).mkdir(parents=True, exist_ok = True)

epochs = 20
batch_size =64

train_spe = len(os.listdir(dataset+"Training/Real"))//batch_size
print(train_spe)
val_spe = len(os.listdir(dataset+"Validation/Fake"))//batch_size
print(val_spe)
test_spe = len(os.listdir(dataset+"Testing/Fake"))//(batch_size//4)
print(test_spe)

# Train multiple times
attempt=0
print('Attempt #',attempt)
foldername = model_path+"Custom_Dataset"+"_xceptionimagenet_attempt"+str(attempt+1)+"epoch_20/"
Path(foldername).mkdir(parents=True, exist_ok=True)
modelname = foldername+"_xceptionimagenet_"+str(num_images)+"_"+str(img_size)+"_attempt"+str(attempt+1)
train_datagen = video_data_generator(dataset,'Training/', batch_size =batch_size)
val_datagen = video_data_generator(dataset,'Validation/', batch_size = batch_size)
test_datagen = video_data_generator(dataset,'Testing/', batch_size = batch_size)
base_model = Xception2() 
base_model.load_weights('imagenet_1.h5')
base_model.trainable = False
    
inputs = layers.Input(shape=(None,None, None,3))
outputs = base_model(inputs, training=False)
outputs = layers.Dropout(0.4, name = 'lstm_dropout')(outputs)
outputs = layers.LSTM(1, dtype='float32', name='LSTM_False')(outputs)
outputs = (outputs * 0.5) + 0.5
cnnlstm_model = models.Model(inputs, outputs, name='xception_cnnlstm')
cnnlstm_model.compile(optimizer = Adam(1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])
callbacks = get_callbacks(modelname, es_patience = 6, rlr_patience = 4, verbose = 1)
history = cnnlstm_model.fit(train_datagen, verbose = 2, steps_per_epoch =train_spe, epochs = 20, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
best_valloss = callbacks[0].best
cnnlstm_model = load_model(modelname+".h5")
test_datagen = video_data_generator(dataset,'Testing/', batch_size = batch_size)
print(cnnlstm_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))

print('I am best validation loss:')
print(best_valloss)


In [ ]:
#predictions
#50bestvalloss 0.6355091333389282

testy=np.zeros(shape=(256,1))
for i in range (256):
  if(i%2!=0):
    testy[i]=float(1)
  else:
    testy[i]=float(0)

yhat_classes=cnnlstm_model.predict(test_datagen,steps=2*test_spe,verbose=0)

#ROC CURVE
fpr, tpr, thresholds = roc_curve(testy, yhat_classes)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
print('Area under the Receiver Operating Characteristic curve:', roc_auc_score(testy, yhat_classes))

#Confusion MAtrix
cf_matrix=confusion_matrix(testy,yhat_classes.round())

labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Zero', 'One']
make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')
plt.show()
                      
#Traning Validation accuracy as well as losses
summarize_diagnostics(history,20)

#PR curve
precision, recall, thresholds = precision_recall_curve(testy, yhat_classes)
plt.plot(recall, precision)
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')

In [ ]:

'''
# Video
#from tensorflow.keras.applications import Xception
model_path = "/content/drive/My Drive/All_Models/" # Save models here
Path(model_path).mkdir(parents=True, exist_ok = True)

epochs = 30
batch_size =32
attempt=0
print('Attempt #',attempt)
foldername = model_path+"Custom_Dataset"+"_xceptionimagenet_attempt"+str(attempt+1)+"epoch_30/"
Path(foldername).mkdir(parents=True, exist_ok=True)
modelname = foldername+"_xceptionimagenet_"+str(num_images)+"_"+str(img_size)+"_attempt"+str(attempt+1)
train_datagen = video_data_generator(dataset,'Training/', batch_size =batch_size)
val_datagen = video_data_generator(dataset,'Validation/', batch_size = batch_size)
test_datagen = video_data_generator(dataset,'Testing/', batch_size = batch_size)
cnnlstm_model = load_model("/content/drive/MyDrive/All_Models/Custom_Dataset_xceptionimagenet_attempt1epoch_30/_xceptionimagenet_10_128_attempt1.h5")
train_spe = len(os.listdir(dataset+"Training/Real"))//batch_size
print(train_spe)
val_spe = len(os.listdir(dataset+"Validation/Fake"))//batch_size
print(val_spe)
test_spe = len(os.listdir(dataset+"Testing/Fake"))//(batch_size//4)
print(test_spe)
base_model = Xception2() 
base_model.load_weights('dataset/imagenet_1.h5')

best_valloss=0.5887700915336609'''
base_model.trainable = True
cnnlstm_model.compile(optimizer = Adam(1e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])

callbacks = get_callbacks(modelname + '_finetuned', es_patience = 7, rlr_patience = 4, verbose = 1)
callbacks[0].best = best_valloss
history = cnnlstm_model.fit(train_datagen, verbose = 2, steps_per_epoch = train_spe, epochs = 20, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
cnnlstm_model = load_model(modelname+".h5")
test_datagen = video_data_generator(dataset,'Testing/', batch_size = batch_size)
print(cnnlstm_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))

In [ ]:
#predictions
testy=np.zeros(shape=(256,1))
for i in range (256):
  if(i%2!=0):
    testy[i]=float(1)
  else:
    testy[i]=float(0)

yhat_classes=cnnlstm_model.predict(test_datagen,steps=2*test_spe,verbose=0)

#ROC CURVE
fpr, tpr, thresholds = roc_curve(testy, yhat_classes)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
print('Area under the Receiver Operating Characteristic curve:', roc_auc_score(testy, yhat_classes))

#Confusion MAtrix
cf_matrix=confusion_matrix(testy,yhat_classes.round())

labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Zero', 'One']
make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='Blues')
plt.show()
                      
#Traning Validation accuracy as well as losses
summarize_diagnostics(history,20)

#PR curve
precision, recall, thresholds = precision_recall_curve(testy, yhat_classes)
plt.plot(recall, precision)
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')